<a href="https://colab.research.google.com/github/LuigiSigillo/Aviation-accidents-visualizations/blob/main/Data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data preprocessing

before preprocessing

In [2]:
import sys
import time
import os
import pandas as pd
from google.colab import drive

drive.mount('/content/drive')
#!unzip "/content/drive/My Drive/AviationData.csv.zip" -d "/content/drive/My Drive/"

df = pd.read_csv("/content/drive/My Drive/Aviation cleaned.csv", encoding='ISO-8859-1')
df

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Event.Id,Event.Date,Location,Aircraft.Damage,Make,Purpose.of.Flight,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.Phase.of.Flight,Crash.Location
0,20200502X81549,2020-05-02,"PALMYRA, IL",Destroyed,Yakovlev,Personal,1.0,0.0,0.0,0.0,VMC,MANEUVERING,"39.408889,-89.990277"
1,20200413X13054,2020-04-11,"Eagle River, AK",Substantial,Cessna,Personal,0.0,2.0,2.0,0.0,VMC,MANEUVERING,"61.351943999999996,-149.548889"
2,20200326X82317,2020-03-25,"Waxahachie, TX",Substantial,Cessna,Personal,0.0,1.0,0.0,0.0,VMC,STANDING,"32.448055,-96.913889"
3,20200324X34735,2020-03-23,"Swansboro, NC",Destroyed,Maule,Personal,2.0,0.0,0.0,0.0,IMC,UNKNOWN,"34.416945,-77.034166"
4,20200318X22054,2020-03-18,"Eagle Creek, OR",Destroyed,Piper,Personal,0.0,1.0,0.0,0.0,VMC,TAKEOFF,"45.352778,-122.340833"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2623,20010417X00765,2001-04-09,"Vero Beach, FL",Destroyed,Piper,Personal,2.0,0.0,0.0,0.0,VMC,TAKEOFF,"27.649721999999997,-80.439166"
2624,20010206X00396,2001-01-24,"Nashville, TN",Destroyed,Beech,Business,4.0,0.0,0.0,0.0,VMC,TAKEOFF,"36.13,-86.653889"
2625,20010119X00328,2001-01-14,"Troy, AL",Destroyed,Learjet,Executive/Corporate,0.0,2.0,0.0,0.0,VMC,LANDING,"31.866667,-86.016667"
2626,20001212X20873,2000-04-05,"MARIANNA, FL",Destroyed,LEARJET,Instructional,3.0,0.0,0.0,0.0,VMC,APPROACH,"1.483333,-75.533333"


# After preprocessing

In [ ]:
import numpy as np
df = pd.read_csv("/content/drive/My Drive/AviationData.csv", encoding='ISO-8859-1')

#View the dataset
print("1, intero", df.shape, "TOT = ", df.shape[0]*df.shape[1])



df = df.drop(
    columns=["Accident.Number","Airport.Code","Airport.Name","Registration.Number","Engine.Type","FAR.Description","Schedule","Air.Carrier","Report.Status","Publication.Date"]
)
print("2, senza colonne inutili", df.shape, "TOT = ", df.shape[0]*df.shape[1])



print("__________________________________________")
df = df.drop(df[df["Investigation.Type"] == "Incident"].index)
df=df.drop(columns=["Investigation.Type"])

print("3, senza incidents", df.shape, "TOT = ", df.shape[0]*df.shape[1])



print("__________________________________________")
df = df[df["Latitude"].notnull()]
df = df[df["Longitude"].notnull()]

print("4, solo coordinate", df.shape, "TOT = ", df.shape[0]*df.shape[1])


print("__________________________________________")
## Da mettere solo quelli con fatal (#num morti) o non fatal
patternDel = ".*Fatal.*"
filter = df["Injury.Severity"].str.contains(patternDel)
df = df[filter]

print("5, solo severity", df.shape, "TOT = ", df.shape[0]*df.shape[1])


print("__________________________________________")
df = df[df["Aircraft.Damage"].notnull()]
print("6, solo damage", df.shape, "TOT = ", df.shape[0]*df.shape[1])


print("__________________________________________")
df = df[df["Aircraft.Category"]=="Airplane"]
df=df.drop(columns=["Aircraft.Category"])

print("7, solo airplane", df.shape, "TOT = ", df.shape[0]*df.shape[1])

print("__________________________________________")
df = df[df["Amateur.Built"]=="No"]
df=df.drop(columns=["Amateur.Built"])
print("8, solo non amatoriali", df.shape, "TOT = ", df.shape[0]*df.shape[1])


print("__________________________________________")
df = df[df["Purpose.of.Flight"].notnull()]
df = df[df["Purpose.of.Flight"]!="Unknown"]
print("9, solo con purpose", df.shape, "TOT = ", df.shape[0]*df.shape[1])

print("__________________________________________")
df = df.dropna(subset=['Total.Fatal.Injuries','Total.Serious.Injuries'], how='all')
print("10, tutti NAN a casa", df.shape, "TOT = ", df.shape[0]*df.shape[1])


print("__________________________________________")
df = df[df["Weather.Condition"].notnull()]
print("11, solo con wether", df.shape, "TOT = ", df.shape[0]*df.shape[1])

print("__________________________________________")
df = df[df["Broad.Phase.of.Flight"].notnull()]
print("12, solo board phase", df.shape, "TOT = ", df.shape[0]*df.shape[1])


print("__________________________________________")
df = df[df["Number.of.Engines"].notnull()]
df=df.drop(columns=["Number.of.Engines"])

print("13, solo motore", df.shape, "TOT = ", df.shape[0]*df.shape[1])


print("__________________________________________")
df = df[df["Model"].notnull()]
df = df[df["Make"].notnull()]
print("14, not nan make e model", df.shape, "TOT = ", df.shape[0]*(df.shape[1]-3))



print("__________________________________________")
df =  df.drop(df[df["Country"] != "United States"].index)
df=df.drop(columns=["Country"])
print("15, only USA", df.shape, "TOT = ", df.shape[0]*(df.shape[1]-3))



df=df.drop(columns=["Injury.Severity"])
df=df.drop(columns=["Model"])
df["Crash.Location"] = df["Latitude"].astype(str) + "," + df["Longitude"].astype(str) 
df=df.drop(columns=["Latitude"])
df=df.drop(columns=["Longitude"])
df['Total.Fatal.Injuries'] = df['Total.Fatal.Injuries'].fillna(0)
df = df.fillna(0)


"""
print("__________________________________________")
df = df.drop(df[df["Injury.Severity"] == "Non-Fatal"].index)
df = df.drop(df[df["Injury.Severity"] == "Unavailable"].index)
print("16, solo fatali", df.shape, "TOT = ", df.shape[0]*df.shape[1]-3)
print(df.head())
"""
"""
print("__________________________________________")
df = df.drop(df[df["Event.Date"] < "2015-01-01"].index)
df = df.drop(df[df["Injury.Severity"] == "Unavailable"].index)
print("4, dal 2000 in poi", df.shape, "TOT = ", df.shape[0]*(df.shape[1]-3))
print(df.head())
"""


print("TOT = ", df.shape[0]*(df.shape[1]))
df.to_csv(r'/content/drive/My Drive/Aviation cleaned.csv', index = False)
df.describe()
df

# Coordinates to Country

In [30]:
import requests
from bs4 import BeautifulSoup
apiurl = "http://api.geonames.org/countrySubdivision?"
#lat=42.561944&lng=-71.77166700000001&username=LENZA
lat = "&lat="
lng = "&lng="
user = "&username=LENZA"




for index, row in df.iterrows():
    latlng = row['Crash.Location']

    print(str(index) + " | " + row['Crash.Location'])
    # index -> indice di riga
    # row['Crash.Location'] -> latitudine,longitudine
    latitude = latlng.split(",")[0]
    longitude = latlng.split(",")[1]
    #print(latitude + " | " + longitude)
    r = requests.get(apiurl + lat + latitude + lng + longitude + user)
    print(r)
    soup = BeautifulSoup(r.content, 'html.parser')
    #print(soup.prettify())
    try:
        print(soup.adminname1.string)
        df.loc[index, 'Crash.Country'] = soup.adminname1.string
    except:
        df.loc[index, 'Crash.Country'] = "Unknown"
    #df[index]['Crash.Country'] = soup.adminname1.string
    

df


Output streaming troncato alle ultime 5000 righe.
California
944 | 32.615,-96.5425
<Response [200]>
Texas
945 | 47.600833,-114.210278
<Response [200]>
Montana
946 | 38.616667,-77.625834
<Response [200]>
Virginia
947 | 37.218056,-93.265833
<Response [200]>
Missouri
948 | 40.718610999999996,-114.02861100000001
<Response [200]>
Utah
949 | 39.180278,-77.18055600000001
<Response [200]>
Maryland
950 | 40.451944,-105.011389
<Response [200]>
Colorado
951 | 35.12,-89.24
<Response [200]>
Tennessee
952 | 32.556389,-114.69
<Response [200]>
Arizona
953 | 34.820555999999996,-87.04388900000001
<Response [200]>
Alabama
954 | 38.935277,-92.684166
<Response [200]>
Missouri
955 | 45.202222,-104.22555600000001
<Response [200]>
Montana
956 | 41.951111,-86.36805600000001
<Response [200]>
Michigan
957 | 30.345833000000002,-97.20666700000001
<Response [200]>
Texas
958 | 40.076667,-123.82055600000001
<Response [200]>
California
959 | 38.8375,-92.008889
<Response [200]>
Missouri
960 | 33.183334,-84.371667
<Resp

,Event.Id,Event.Date,Location,Aircraft.Damage,Make,Purpose.of.Flight,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.Phase.of.Flight,Crash.Location,Crash.Country
0,20200502X81549,2020-05-02,"PALMYRA, IL",Destroyed,Yakovlev,Personal,1.0,0.0,0.0,0.0,VMC,MANEUVERING,"39.408889,-89.990277",Illinois
1,20200413X13054,2020-04-11,"Eagle River, AK",Substantial,Cessna,Personal,0.0,2.0,2.0,0.0,VMC,MANEUVERING,"61.351943999999996,-149.548889",Alaska
2,20200326X82317,2020-03-25,"Waxahachie, TX",Substantial,Cessna,Personal,0.0,1.0,0.0,0.0,VMC,STANDING,"32.448055,-96.913889",Texas
3,20200324X34735,2020-03-23,"Swansboro, NC",Destroyed,Maule,Personal,2.0,0.0,0.0,0.0,IMC,UNKNOWN,"34.416945,-77.034166",Unknown
4,20200318X22054,2020-03-18,"Eagle Creek, OR",Destroyed,Piper,Personal,0.0,1.0,0.0,0.0,VMC,TAKEOFF,"45.352778,-122.340833",Oregon
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2623,20010417X00765,2001-04-09,"Vero Beach, FL",Destroyed,Piper,Personal,2.0,0.0,0.0,0.0,VMC,TAKEOFF,"27.649721999999997,-80.439166",Florida
2624,20010206X00396,2001-01-24,"Nashville, TN",Destroyed,Beech,Business,4.0,0.0,0.0,0.0,VMC,TAKEOFF,"36.13,-86.653889",Tennessee
2625,20010119X00328,2001-01-14,"Troy, AL",Destroyed,Learjet,Executive/Corporate,0.0,2.0,0.0,0.0,VMC,LANDING,"31.866667,-86.016667",Alabama
2626,20001212X20873,2000-04-05,"MARIANNA, FL",Destroyed,LEARJET,Instructional,3.0,0.0,0.0,0.0,VMC,APPROACH,"1.483333,-75.533333",Caquetá


In [31]:
df

,Event.Id,Event.Date,Location,Aircraft.Damage,Make,Purpose.of.Flight,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.Phase.of.Flight,Crash.Location,Crash.Country
0,20200502X81549,2020-05-02,"PALMYRA, IL",Destroyed,Yakovlev,Personal,1.0,0.0,0.0,0.0,VMC,MANEUVERING,"39.408889,-89.990277",Illinois
1,20200413X13054,2020-04-11,"Eagle River, AK",Substantial,Cessna,Personal,0.0,2.0,2.0,0.0,VMC,MANEUVERING,"61.351943999999996,-149.548889",Alaska
2,20200326X82317,2020-03-25,"Waxahachie, TX",Substantial,Cessna,Personal,0.0,1.0,0.0,0.0,VMC,STANDING,"32.448055,-96.913889",Texas
3,20200324X34735,2020-03-23,"Swansboro, NC",Destroyed,Maule,Personal,2.0,0.0,0.0,0.0,IMC,UNKNOWN,"34.416945,-77.034166",Unknown
4,20200318X22054,2020-03-18,"Eagle Creek, OR",Destroyed,Piper,Personal,0.0,1.0,0.0,0.0,VMC,TAKEOFF,"45.352778,-122.340833",Oregon
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2623,20010417X00765,2001-04-09,"Vero Beach, FL",Destroyed,Piper,Personal,2.0,0.0,0.0,0.0,VMC,TAKEOFF,"27.649721999999997,-80.439166",Florida
2624,20010206X00396,2001-01-24,"Nashville, TN",Destroyed,Beech,Business,4.0,0.0,0.0,0.0,VMC,TAKEOFF,"36.13,-86.653889",Tennessee
2625,20010119X00328,2001-01-14,"Troy, AL",Destroyed,Learjet,Executive/Corporate,0.0,2.0,0.0,0.0,VMC,LANDING,"31.866667,-86.016667",Alabama
2626,20001212X20873,2000-04-05,"MARIANNA, FL",Destroyed,LEARJET,Instructional,3.0,0.0,0.0,0.0,VMC,APPROACH,"1.483333,-75.533333",Caquetá


In [32]:
print("TOT = ", df.shape[0]*(df.shape[1]))
df.to_csv(r'/content/drive/My Drive/AviationCrashLocation.csv', index = False)
df.describe()
df

TOT =  36792


,Event.Id,Event.Date,Location,Aircraft.Damage,Make,Purpose.of.Flight,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.Phase.of.Flight,Crash.Location,Crash.Country
0,20200502X81549,2020-05-02,"PALMYRA, IL",Destroyed,Yakovlev,Personal,1.0,0.0,0.0,0.0,VMC,MANEUVERING,"39.408889,-89.990277",Illinois
1,20200413X13054,2020-04-11,"Eagle River, AK",Substantial,Cessna,Personal,0.0,2.0,2.0,0.0,VMC,MANEUVERING,"61.351943999999996,-149.548889",Alaska
2,20200326X82317,2020-03-25,"Waxahachie, TX",Substantial,Cessna,Personal,0.0,1.0,0.0,0.0,VMC,STANDING,"32.448055,-96.913889",Texas
3,20200324X34735,2020-03-23,"Swansboro, NC",Destroyed,Maule,Personal,2.0,0.0,0.0,0.0,IMC,UNKNOWN,"34.416945,-77.034166",Unknown
4,20200318X22054,2020-03-18,"Eagle Creek, OR",Destroyed,Piper,Personal,0.0,1.0,0.0,0.0,VMC,TAKEOFF,"45.352778,-122.340833",Oregon
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2623,20010417X00765,2001-04-09,"Vero Beach, FL",Destroyed,Piper,Personal,2.0,0.0,0.0,0.0,VMC,TAKEOFF,"27.649721999999997,-80.439166",Florida
2624,20010206X00396,2001-01-24,"Nashville, TN",Destroyed,Beech,Business,4.0,0.0,0.0,0.0,VMC,TAKEOFF,"36.13,-86.653889",Tennessee
2625,20010119X00328,2001-01-14,"Troy, AL",Destroyed,Learjet,Executive/Corporate,0.0,2.0,0.0,0.0,VMC,LANDING,"31.866667,-86.016667",Alabama
2626,20001212X20873,2000-04-05,"MARIANNA, FL",Destroyed,LEARJET,Instructional,3.0,0.0,0.0,0.0,VMC,APPROACH,"1.483333,-75.533333",Caquetá
